# Games RSs

In [1]:
# импорты, которые точно понадобятся
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
from scipy.sparse import random
from scipy.sparse import save_npz

In [3]:
from scipy.sparse import load_npz
from scipy.sparse import vstack

In [4]:
# Данные взяты отсюда - http://jmcauley.ucsd.edu/data/amazon/
# http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Video_Games_5.json.gz
JSON_DATA_PATH = "lab_data/Video_Games_5.json"
N = 10

## Анализ данных

In [5]:
import json

def iter_json_data(path):
    with open(path) as f:
        for line in f:
            data = json.loads(line)
            yield data
            
def get_data_frame():
    uid_to_id = {}
    iid_to_id = {}
    
    cols = ["uid", "iid", "review", "rating", "dt"]
    rows = []
    for d in iter_json_data(JSON_DATA_PATH):
        uid = uid_to_id.setdefault(d["reviewerID"], len(uid_to_id))
        iid = iid_to_id.setdefault(d["asin"], len(iid_to_id))
        review = d["reviewText"]
        rating = float(d["overall"])
        dt = int(d["unixReviewTime"])
        rows.append((uid, iid, review, rating, dt))
        
        
    return pd.DataFrame(rows, columns=cols)

In [6]:
df = get_data_frame()
df.head()

,uid,iid,review,rating,dt
0,0,0,Installing the game was a struggle (because of...,1.0,1341792000
1,1,0,If you like rally cars get this game you will ...,4.0,1372550400
2,2,0,1st shipment received a book instead of the ga...,1.0,1403913600
3,3,0,"I got this version instead of the PS3 version,...",3.0,1315958400
4,4,0,I had Dirt 2 on Xbox 360 and it was an okay ga...,4.0,1308009600


In [7]:
print("min-max количество объектов на пользователя:", 
      df.groupby("uid").iid.nunique().min(), df.groupby("uid").iid.nunique().max())
print("min-max количество пользователей на объект:", 
      df.groupby("iid").uid.nunique().min(), df.groupby("iid").uid.nunique().max())

min-max количество объектов на пользователя: 5 773
min-max количество пользователей на объект: 5 802


In [8]:
# проверяем, есть ли случаи, когда один и тот же пользователь оставляет отзывы на один и тот же объект
df.groupby(["uid", "iid"]).review.count().unique()  # ура, таких случаев нет

array([1], dtype=int64)

In [9]:
print("Количество объектов:", df.iid.unique().size)
print("Количество пользователей:", df.uid.unique().size)

Количество объектов: 10672
Количество пользователей: 24303


## Готовим выборки

In [10]:
def split_df_by_dt(df, p=0.8):
    """Функция разбивает df на тестовую и тренировочную выборки по времени 
    публикации отзывов (значение времени в поле dt)
    
    :param p: персентиль значений dt, которые образуют тренировочную выборку. Например p=0.8 означает, что в 
    тренировочной части будут отзывы, соответствующие первым 80% временного интервала 
    :return: два pd.DataFrame объекта
    """
    border_dt = df.dt.quantile(p)
    print("Min=%s, border=%s, max=%s" % (df.dt.min(), border_dt, df.dt.max()))
    training_df, test_df  = df[df.dt <= border_dt], df[df.dt > border_dt]
    print("Размер до очистки:", training_df.shape, test_df.shape)
    # удаляем из тестовых данных строки, соответствующие пользователям или объектам, 
    # которых нет в тренировочных данных 
    # (пользователи - избегаем проблем для персональных систем, объекты - для всех)
    test_df = test_df[test_df.uid.isin(training_df.uid) & test_df.iid.isin(training_df.iid)]
    print("Размер после очистки:", training_df.shape, test_df.shape)
    return training_df, test_df

In [11]:
training_df, test_df = split_df_by_dt(df, p=0.8)
del df

Min=939859200, border=1377129600.0, max=1405987200
Размер до очистки: (185427, 5) (46353, 5)
Размер после очистки: (185427, 5) (19174, 5)


In [12]:
def clean_df(df, min_review_per_uid, min_review_per_iid):
    """Функция удаляет из df строки, соответствующие пользователям и объектам, 
    у которых меньше min_review_per_uid и min_review_per_iid отзывов соответственно
    """
    _df = df.copy()
    while True:
        review_per_uid = _df.groupby("uid").review.count()
        bad_uids = review_per_uid[review_per_uid < min_review_per_uid].index
    
        review_per_iid = _df.groupby("iid").review.count()
        bad_iids = review_per_iid[review_per_iid < min_review_per_iid].index
        
        if bad_uids.shape[0] > 0 or bad_iids.shape[0] > 0:
            _df = _df[(~_df.uid.isin(bad_uids)) & (~_df.iid.isin(bad_iids))]
        else:
            break
    return _df

 ## Метрика

In [13]:
def hit_ratio(recs_dict, test_dict):
    """Функция считает метрику hit-ration для двух словарей
    :recs_dict: словарь рекомендаций типа {uid: {iid: score, ...}, ...}
    :test_dict: тестовый словарь типа {uid: {iid: score, ...}, ...}
    """
    hits = 0
    for uid in test_dict:
        if set(test_dict[uid].keys()).intersection(recs_dict.get(uid, {})):
            hits += 1
    return hits / len(test_dict)

In [14]:
def get_test_dict(test_df):
    """Функция, конвертирующая тестовый df в словарь
    """
    test_dict = {}
    for t in test_df.itertuples():
        test_dict.setdefault(t.uid, {})
        test_dict[t.uid][t.iid] = t.rating
    return test_dict

test_dict = get_test_dict(test_df)

## Базовые классы для рекомендательной системы

In [20]:
class BasicRecommender(object):
    def __init__(self):
        pass
    
    def get_recs(self, uid, top):
        """Строит рекомендации для пользователя uid
        :return: словарь типа {iid: score, ...}
        """
        return {}
    
    def get_batch_recs(self, uids, top):
        """Строит рекомендации для нескольких пользователей uids
        :return: словарь типа {uid: {iid: score, ...}, ...}
        """
        return {uid: self.get_recs(uid, top) for uid in uids}
    
class NonPersRecommender(BasicRecommender):
    
    def __init__(self, df):
        super(NonPersRecommender, self).__init__()
        self.recs = self._prepare_recs(df)
        
    def _prepare_recs(self, df):
        return pd.Series([])
    
    def get_recs(self, uid, top):
        from collections import OrderedDict
        return OrderedDict(self.recs[:top])
    
    def get_batch_recs(self, uids, top):
        non_pers_recs = self.get_recs(None, top)
        return {uid: non_pers_recs for uid in uids}

## Векторизуем объекты и строим рекомендации

In [23]:
item_ids = training_df.iid.unique() # уникальные идентификаторы объектов в тренировочной выборке
id_to_ind = {}
texts = []

# для каждого id из item_ids формируется "суммарное" описание из отзывово нескольких пользователей
for id in item_ids:
    text = training_df[training_df.iid == id].review.str.cat(sep=' ')
    texts.append(text)
    id_to_ind[id] = id_to_ind.setdefault(id, len(id_to_ind)) 

In [84]:
class ContentBasedRecommender(NonPersRecommender):
    
    def __init__(self, text_array, iid_to_ind, train_df, max_features=None, use_dt=True, use_ratings=False):
        super(NonPersRecommender, self).__init__()

        from sklearn.feature_extraction.text import TfidfVectorizer
        import numpy as np
        import pandas as pd
        

        self.items_matrix = TfidfVectorizer(stop_words='english', max_features=max_features).fit_transform(text_array)
        self.iid_to_ind = iid_to_ind
        self.df = train_df[['iid','uid', 'dt', 'rating']]
        self.use_ratings = use_ratings
        self.use_dt = use_dt
        self.DICT_LEN = self.items_matrix.shape[1]
        self.uinque_ids = train_df.iid.unique()
        
    def get_recs(self, uids_to_recommend, top_k=10):
        self.recs = self._prepare_recs(uids_to_recommend, top_k)
        return self.recs

    def valid_ids(bought_ids):
        return set(self.unique_ids).difference(set(bought_ids))
    
    def select_item(source_matrix, keys_to_index, key_, exclude_keys):
        if key_ in exclude_keys:
            return 0
        else:
            return source_matrix[0, keys_to_index[key_]]
        
    def _prepare_recs(self, uids_to_recommend, top_k):
        from sklearn.metrics.pairwise import cosine_similarity
        recs = {}
        for uid in uids_to_recommend:
            user_df = self.df[self.df.uid == uid]
            user_items = user_df.iid

            if self.use_dt:
                user_dt = user_df.dt
                user_dt = user_dt/max(user_dt)
            else:
                user_dt = np.ones(len(user_items))

            if self.use_ratings:
                user_ratings = user_df.rating
            else:
                user_ratings = np.ones(len(user_items))

            #инициализируем вектор плбьзователя
            user_vect = csr_matrix((1, self.DICT_LEN))

            #вектор пользователя
            for iid, time, rating in zip(user_items, user_dt, user_ratings):
                item_vect = self.items_matrix.getrow(self.iid_to_ind[iid])
                user_vect += item_vect.multiply(time*rating)

            #вектор схожести для пользователя
            sim_vector = cosine_similarity(user_vect, self.items_matrix, dense_output=True)
            item_with_score = { iid: sim_vector[0, self.iid_to_ind[iid]] for iid in set(self.uinque_ids).difference(set(user_items)) }
            user_recs = sorted([(k,v) for k,v in item_with_score.items()], key=lambda x: -x[1])[:top_k]
            user_recs = dict(user_recs)
            recs[uid] = user_recs

        return recs

In [85]:
recommender = ContentBasedRecommender(texts, id_to_ind, training_df, use_dt=False, use_ratings=False)

In [87]:
recs = recommender.get_recs(test_dict.keys())

In [61]:
hit_ratio(recs, test_dict)

0.082